In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import torch
import torch.nn as nn
import time
import pickle

In [2]:
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import top_k_accuracy_score

In [3]:
sys.path.append('/global/u2/v/vboehm/codes/SIG_GIS/')
from sig_gis import *
from sig_gis.GIS import *

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda', index=0)

In [6]:
RUN             = '1'
EPOCHS          = 40

seeds           = {'1':512, '2':879, '3':9981, '4': 20075, '5': 66, '6': 276, '7': 936664}

# conditional     = False
# cond_on         = 'type'

root_model_data = '/global/cscratch1/sd/vboehm/Datasets/sdss/by_model/'
# root_models     = '/global/cscratch1/sd/vboehm/Models/SDSS_AE/'
# root_encoded    = '/global/cscratch1/sd/vboehm/Datasets/encoded/sdss/'
# root_decoded    = '/global/cscratch1/sd/vboehm/Datasets/decoded/sdss/'


# wlmin, wlmax    = (3388,8318)
# fixed_num_bins  = 1000
# min_SN          = 50
# min_z           = 0.05
# max_z           = 0.36
# label           = 'galaxies_quasars_bins%d_wl%d-%d'%(fixed_num_bins,wlmin,wlmax)
# label_          = label+'_minz%s_maxz%s_minSN%d'%(str(int(min_z*100)).zfill(3),str(int(max_z*100)).zfill(3),min_SN)
# label_2         = label_+'_10_fully_connected_mean_div_spec_norm'

# if conditional:
#     label_2='conditional_%s'%cond_on+label_2

# upsampling      = 'SMOTE'
# fac             = 5

data_path       = '/global/cscratch1/sd/vboehm/SDSSOutlier/fc'

In [7]:
torch.manual_seed(seeds[RUN])
np.random.seed(seeds[RUN])

In [8]:
encoded_train, encoded_valid = np.load(os.path.join(data_path,'encoded_fc.npy'), allow_pickle=True)

In [9]:
train = pickle.load(open(os.path.join(root_model_data,'SDSS_DR16_preprocessed_train.pkl'),'rb'))
valid = pickle.load(open(os.path.join(root_model_data,'SDSS_DR16_preprocessed_valid.pkl'),'rb'))

In [14]:
train_short ={}
train_short['spec']=train['spec'][:100000]
train_short['noise']=train['noise'][:100000]
train_short['mask']=train['mask'][:100000]

In [17]:
pickle.dump(train_short,open(os.path.join(root_model_data,'DR16_train.npy'),'wb'))

In [19]:
root_model_data

'/global/cscratch1/sd/vboehm/Datasets/sdss/by_model/'

In [16]:
test_short ={}
test_short['spec']=valid['spec'][:30000]
test_short['noise']=valid['noise'][:30000]
test_short['mask']=valid['mask'][:30000]

In [18]:
pickle.dump(test_short,open(os.path.join(root_model_data,'DR16_test.npy'),'wb'))

In [12]:
len(valid['spec']), len(encoded_valid)

(139642, 139642)

In [13]:
for data in train, valid:
    sub_class = data['subclass']
    gal_type  = data['class']
    broadline = []
    subclass  = []
    gal_class = []
    
    for ii in range(len(data['subclass'])):
        if gal_type[ii]==2:
            gal_class.append(0)
        elif gal_type[ii]==1:
            gal_class.append(1)     
        else:
            print('unrecognized subclass')
        if sub_class[ii] in [0,3]:
            subclass.append(0)
        elif sub_class[ii] in [1,2]:
            subclass.append(1)
        elif sub_class[ii] in [4,5]:
            subclass.append(2)
        elif sub_class[ii] in [6,7]:
            subclass.append(3)
        else:
            print('unrecognized subclass')
        if sub_class[ii] in [2,3,5,7]:
            broadline.append(1)
        elif sub_class[ii] in [0,1,4,6]:
            broadline.append(0)
        else:
            print('unrecognized subclass')    
            
    broadline = np.asarray(broadline)
    subclass  = np.asarray(subclass)
    gal_class = np.asarray(gal_class)
    
    data['labels'] = np.vstack([gal_class,subclass,broadline])

In [14]:
le_dict0 = {0:'GAL',1: 'QSO'}
le_dict1 = {0:'NO SUBCLASS',1: 'AGN', 2:'STARBURST', 3:'STARFORMING'}
le_dict2 = {0:'NOT BROADLINE', 1:'BROADLINE'}

In [15]:
def names(X, dicts):
    N,c = X.shape
    assert(len(dicts)==c)
    names = []
    for ii in range(N):
        name = ''
        for jj in range(len(dicts)):
            if jj==0:
                name=''.join([name,dicts[jj][X[ii][jj]]])
            else:
                #print(dicts[jj][X[ii][jj]])
                name=' '.join([name,dicts[jj][X[ii][jj]]])
        names.append(name)
    return names

In [16]:
def flatten(X):
    N,c = X.shape
    print(N,c)
    classes = []
    counts = []
    class_names = []
    for ii in range(c):
        counts.append(len(np.unique(X[:,ii])))
    n_classes = np.prod(counts)
    print(n_classes, counts)
    for ii in range(N):
        class_ = 0
        for jj in np.arange((c)):
            offset = n_classes/np.prod(counts[0:(jj+1)])
            class_+=X[ii][jj]*offset
        classes.append(class_)
        
    return classes

In [17]:
res   = flatten(train['labels'].T)
names = names(train['labels'].T,[le_dict0,le_dict1,le_dict2])

209462 3
16 [2, 4, 2]


In [18]:
df = pd.DataFrame()
df['names']  = names
df['labels'] = res
df = df[['names', 'labels']].drop_duplicates().reset_index().drop(columns=['index'])

# le_new = pd.Series(df.names.values,index=df.labels).to_dict()
# pickle.dump(le_new, open('new_inflated_labels.pkl','wb'))

In [19]:
for data in [train,valid]:
    data['new_inf_labels'] = flatten(data['labels'].T)

209462 3
16 [2, 4, 2]
139642 3
16 [2, 4, 2]


In [20]:
pickle.dump(train,open(os.path.join(root_model_data,'SDSS_DR16_preprocessed_train_labeled.pkl'),'wb'))
pickle.dump(valid,open(os.path.join(root_model_data,'SDSS_DR16_preprocessed_valid_labeled.pkl'),'wb'))

In [21]:
train_x = torch.Tensor(encoded_train).to(device) 
train_y = torch.Tensor(train['new_inf_labels']).to(torch.long).to(device)

valid_x = torch.Tensor(encoded_valid).to(device) 
valid_y = torch.Tensor(valid['new_inf_labels']).to(torch.long).to(device)

In [52]:
def _conditional_transform_batch_model(model, data, label, logj, index, batchsize, start_index=0, end_index=None, start=0, end=None, param=None, nocuda=False):

    if torch.cuda.is_available() and not nocuda:
        gpu    = index % torch.cuda.device_count()
        device = torch.device('cuda:%d'%gpu)
    else:
        device = torch.device('cpu')

    model = model.to(device)

    if end_index is None:
        end_index = len(data)

    i = 0
    while i * batchsize < end_index-start_index:
        start_index0 = start_index + i * batchsize 
        end_index0 = min(start_index + (i+1) * batchsize, end_index) 
        if param is None:
            data1, logj1 = model.transform(data[start_index0:end_index0].to(device), label[start_index0:end_index0].to(device), start=start, end=end, param=param)
        else:
            data1, logj1 = model.transform(data[start_index0:end_index0].to(device), label[start_index0:end_index0].to(device), start=start, end=end, param=param[start_index0:end_index0].to(device))
        data[start_index0:end_index0] = data1.to(data.device)
        logj[start_index0:end_index0] = logj[start_index0:end_index0] + logj1.to(logj.device)
        i += 1

    del data1, logj1, model 
    if torch.cuda.is_available() and not nocuda:
        torch.cuda.empty_cache()

    return


def conditional_transform_batch_model(model, data, label, batchsize, logj=None, start=0, end=None, param=None, pool=None, nocuda=False):

    if logj is None:
        logj = torch.zeros(len(data), device=data.device)

    if pool is None: 
        _transform_batch_model(model, data, label, logj, 0, batchsize, start=start, end=end, param=param, nocuda=nocuda) 
    else:
        if torch.cuda.is_available() and not nocuda:
            nprocess = torch.cuda.device_count()
        else:
            nprocess = mp.cpu_count()
        param0 = [(model, data, label, logj, i, batchsize, len(data)*i//nprocess, len(data)*(i+1)//nprocess, start, end, param, nocuda) for i in range(nprocess)]
        pool.starmap(_conditional_transform_batch_model, param0)

    return data, logj


def _conditional_transform_batch_layer(layer, data, label, logj, index, batchsize, start_index=0, end_index=None, direction='forward', param=None, nocuda=False):

    if torch.cuda.is_available() and not nocuda:
        gpu = index % torch.cuda.device_count()
        device = torch.device('cuda:%d'%gpu)
    else:
        device = torch.device('cpu')
    
    layer = layer.to(device)

    if end_index is None:
        end_index = len(data)

    i = 0
    while i * batchsize < end_index-start_index:
        start_index0 = start_index + i * batchsize 
        end_index0 = min(start_index + (i+1) * batchsize, end_index) 
        if direction == 'forward': 
            if param is None:
                data1, logj1 = layer.forward(data[start_index0:end_index0].to(device),label[start_index0:end_index0].to(device), param=param)
            else:
                data1, logj1 = layer.forward(data[start_index0:end_index0].to(device), label[start_index0:end_index0].to(device), param=param[start_index0:end_index0].to(device))
        else: 
            if param is None:
                data1, logj1 = layer.inverse(data[start_index0:end_index0].to(device),label[start_index0:end_index0].to(device), param=param)
            else:
                data1, logj1 = layer.inverse(data[start_index0:end_index0].to(device),label[start_index0:end_index0].to(device), param=param[start_index0:end_index0].to(device))
        data[start_index0:end_index0] = data1.to(data.device)
        logj[start_index0:end_index0] = logj[start_index0:end_index0] + logj1.to(logj.device)
        i += 1

    del data1, logj1, layer 
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return


def conditional_transform_batch_layer(layer, data, label, batchsize, logj=None, direction='forward', param=None, pool=None, nocuda=False):
    assert direction in ['forward', 'inverse']
    
    if logj is None:
        logj = torch.zeros(len(data), device=data.device)
    
    if pool is None: 
        _conditional_transform_batch_layer(layer, data, label, logj, 0, batchsize, direction=direction, param=param, nocuda=nocuda) 
    else:
        if torch.cuda.is_available() and not nocuda:
            nprocess = torch.cuda.device_count()
        else:
            nprocess = mp.cpu_count()
        param0 = [(layer, data, label, logj, i, batchsize, len(data)*i//nprocess, len(data)*(i+1)//nprocess, direction, param, nocuda) for i in range(nprocess)]
        pool.starmap(_conditional_transform_batch_layer, param0)
    
    return data, logj


In [53]:
class ConditionalSINF(nn.Module):

    #Sliced Iterative Normalizing Flow model
    
    def __init__(self, ndim, n_class):
        
        super().__init__()
        
        self.layer = nn.ModuleList([])
        self.ndim = ndim
        self.n_class = n_class
    
    def forward(self, data, label, start=0, end=None):
        
        if data.ndim == 1:
            data = data.view(1,-1)
        if end is None:
            end = len(self.layer)
        elif end < 0:
            end += len(self.layer)
        if start < 0:
            start += len(self.layer)
        
        assert start >= 0 and end >= 0 and end >= start

        logj = torch.zeros(data.shape[0], device=data.device)
        
        for i in range(start, end):
            data, log_j = self.layer[i](data, param=label)
            logj += log_j

        return data, logj
    
    
    def inverse(self, data, label, start=None, end=0, d_dz=None):

        if data.ndim == 1:
            data = data.view(1,-1)
        if end < 0:
            end += len(self.layer)
        if start is None:
            start = len(self.layer)
        elif start < 0:
            start += len(self.layer)
        
        assert start >= 0 and end >= 0 and end <= start

        logj = torch.zeros(data.shape[0], device=data.device)
        
        for i in reversed(range(end, start)):
            if d_dz is None:
                data, log_j = self.layer[i].inverse(data, param=label)
            else:
                data, log_j, d_dz = self.layer[i].inverse(data,d_dz=d_dz, param=label)
            logj += log_j

        if d_dz is None:
            return data, logj
        else:
            return data, logj, d_dz


    def transform(self, data, label, start, end,):

        if start is None:
            return self.inverse(data=data, start=start, end=end, param=label) 
        elif end is None:
            return self.forward(data=data, start=start, end=end, param=label) 
        elif start < 0:
            start += len(self.layer)
        elif end < 0:
            end += len(self.layer)
        
        if start < 0:
            start = 0
        elif start > len(self.layer):
            start = len(self.layer)
        if end < 0:
            end = 0
        elif end > len(self.layer):
            end = len(self.layer)

        if start <= end:
            return self.forward(data=data, start=start, end=end, param=label) 
        else:
            return self.inverse(data=data, start=start, end=end, param=label) 
    
    
    def add_layer(self, layer, position=None):
        
        if position is None or position == len(self.layer):
            self.layer.append(layer)
        else:
            if position < 0:
                position += len(self.layer)
            assert position >= 0 and position < len(self.layer)
            self.layer.insert(position, layer)
        
        return self
    
    
    def delete_layer(self, position=-1):
        
        if position == -1 or position == len(self.layer)-1:
            self.layer = self.layer[:-1]
        else:
            if position < 0:
                position += len(self.layer)
            assert position >= 0 and position < len(self.layer)-1
            
            for i in range(position, len(self.layer)-1):
                self.layer._modules[str(i)] = self.layer._modules[str(i + 1)]
            self.layer = self.layer[:-1]
        
        return self
    
    
    def evaluate_density(self, data, label, start=0, end=None):
        
        data, logj = self.forward(data, label, start=start, end=end)
        logq = -self.ndim/2*torch.log(torch.tensor(2*math.pi)) - torch.sum(data.reshape(len(data), self.ndim)**2,  dim=1)/2
        logp = logj + logq
        
        return logp


    def loss(self, data, start=0, end=None, param=None):
        return -torch.mean(self.evaluate_density(data, label, start=start, end=end, param=param))
    
    
    def sample(self, nsample, start=None, end=0, device=torch.device('cuda'), param=None):

        #device must be the same as the device of the model
        
        x       = torch.randn(nsample, self.ndim, device=device)
        logq    = -self.ndim/2.*torch.log(torch.tensor(2.*math.pi)) - torch.sum(x**2,  dim=1)/2
        x, logj = self.inverse(x, start=start, end=end, param=param)
        logp    = logj + logq

        return x, logp


    def score(self, data, label, start=0, end=None, param=None):

        #returns score = dlogp / dx

        data.requires_grad_(True)
        logp  = torch.sum(self.evaluate_density(data, label, start, end, param))
        score = torch.autograd.grad(logp, data)[0]
        data.requires_grad_(False)

        return score

In [54]:
def train_ConditionalGIS(data_train, label_train, data_valid, label_valid, iteration=None, K=None, M=None, KDE=True, b_factor=1, alpha=None, bounds=None,max_iter=400,
        edge_bins=None, ndata_A=None, MSWD_max_iter=None, NBfirstlayer=False, Whiten=True, batchsize=None, nocuda=False, patch=False, shape=None, model=None, verbose=True):
    
    '''
    data_train: (ndata_train, ndim).
    data_valid: (ndata_valid, ndim), optional. If provided, its logp will be used to determine the number of iterations.
    iteration: integer, optional. The maximum number of GIS iterations. Required if data_valid is not provided.
    K: integer, optional. The number of slices for each iteration. See max K-SWD in the SINF paper. 1 <= K <= ndim.
    M: integer, optional. The number of spline knots for rational quadratic splines.
    KDE: bool. Whether to use KDE for estimating 1D PDF. Recommended True.
    b_factor: positive float number, optional. The multiplicative factor for KDE kernel width.
    alpha: two non-negative float number in the format of (alpha1, alpha2), optional. Regularization parameter. See Equation 13 of SINF paper. alpha1 for interpolation, alpha2 for extrapolation slope. 0 <= alpha1,2 < 1. If not given, very heavy regularization will be used, which could result in slow training and a large number of iterations.
    bounds: sequence, optional. In the format of [[x1_min, x1_max], [x2_min, x2_max], ..., [xd_min, xd_max]]. Represent infinity and negative infinity with None.
    edge_bins: non-negative integer, optional. The number of spline knots at the boundary.
    ndata_A: positive integer, optional. The number of training data used for fitting A (slice axes).
    MSWD_max_iter: positive integer, optional. The maximum number of iterations for optimizing A (slice axes). See Algorithm 1 of SINF paper. Called L_iter in the paper.
    NBfirstlayer: bool, optional. Whether to use Naive Bayes (no rotation) at the first layer.
    Whiten: bool, optional. Whether to whiten the data before applying GIS.
    batchsize: positive integer, optional. The batch size for transforming the data. Does not change the performance. Only saves the memory. 
    Useful when the data is too large and can't fit in the memory.
    nocuda: bool, optional. Whether to use gpu.
    patch: bool, optional. Whether to use patch-based modeling. Only useful for image datasets.
    shape: sequence, optional. The shape of the image datasets, if patch is enabled.
    model: GIS model, optional. Trained GIS model. If provided, new iterations will be added in the model.
    verbose: bool, optional. Whether to print training information.
    '''

    assert data_valid is not None or iteration is not None
 
    #hyperparameters
    ndim    = data_train.shape[1]
    nclass  = len(np.unique(label_train.cpu().numpy()))
    print(ndim,nclass)

    ndata = len(data_train)

    if M is None:
        M = max(min(200, int(ndata**0.5)), 50)
    if alpha is None:
        alpha = (1-0.02*math.log10(ndata), 1-0.001*math.log10(ndata))#).to(device)
    if bounds is not None:
        assert len(bounds) == ndim
        for i in range(ndim):
            assert len(bounds[i]) == 2
    if edge_bins is None:
        edge_bins = max(int(math.log10(ndata))-1, 0)
    if batchsize is None:
        batchsize = len(data_train)
    if not patch:
        if K is None:
            if ndim <= 8 or ndata / float(ndim) < 20:
                K = ndim
            else:
                K = 8
        if ndata_A is None:
            ndata_A = min(len(data_train), int(math.log10(ndim)*1e5))
        if MSWD_max_iter is None:
            MSWD_max_iter = min(round(ndata) // ndim, 200)
    else:
        assert shape[0] > 4 and shape[1] > 4
        K0 = K
        ndata_A0 = ndata_A
        MSWD_max_iter0 = MSWD_max_iter

        
    best_accuracy = 0
    #device
    device = torch.device("cuda:0" if torch.cuda.is_available() and not nocuda else "cpu")

    
    ### taking out logit transform and replacing log jacobian with zeros 
    logj_train    = torch.zeros(data_train.shape[0],device=device)
    logj_valid    = torch.zeros(data_valid.shape[0],device=device)

    #define the model
    if model is None:
        model = ConditionalSINF(ndim=ndim,n_class=nclass).requires_grad_(False).to(device)
        if data_valid is not None:
            best_logp_valid = -1e10
            best_Nlayer     = 0
            wait            = 0
            maxwait         = 5 

    
    #whiten
    if Whiten:
        layer = whiten(ndim_data=ndim, scale=True, ndim_latent=ndim).requires_grad_(False).to(device)
        layer.fit(data_train)

        data_train, logj_train0 = layer(data_train)
        logj_train += logj_train0

        logp_train = (torch.mean(logj_train) - ndim/2*torch.log(torch.tensor(2*math.pi)) - torch.mean(torch.sum(data_train**2,  dim=1)/2)).item()
        
        if data_valid is not None:
            data_valid, logj_valid0 = layer(data_valid)
            logj_valid += logj_valid0
            logp_valid = (torch.mean(logj_valid) - ndim/2*torch.log(torch.tensor(2*math.pi)) - torch.mean(torch.sum(data_valid**2,  dim=1)/2)).item()
            if logp_valid > best_logp_valid:
                best_logp_valid = logp_valid
                best_Nlayer = len(model.layer)

        model.add_layer(layer)
        if verbose:
            if data_valid is not None:
                print('After whiten logp:', logp_train, logp_valid)
            else:
                print('After whiten logp:', logp_train)

                
                
    data_train    = data_train[None,:].repeat_interleave(nclass, axis=0)
    data_valid    = data_valid[None,:].repeat_interleave(nclass, axis=0)

    logj_train    = logj_train[None,:].repeat_interleave(nclass, axis=0)
    logj_valid    = logj_valid[None,:].repeat_interleave(nclass, axis=0)

    #GIS iterations
    
    ii=0
    while True:
        t = time.time()

        if NBfirstlayer:
            layer = ConditionalSlicedTransport_discrete(ndim=ndim, n_class = nclass, K=ndim, M=M).requires_grad_(False).to(device)
        else:
            layer = ConditionalSlicedTransport_discrete(ndim=ndim, n_class = nclass, K=K, M=M).requires_grad_(False).to(device)

        #fit the layer
        if NBfirstlayer:
            layer.A[:] = torch.eye(ndim).to(device)
            NBfirstlayer = False
        elif ndim > 1:
            layer.fit_A(data_train[label_train, torch.arange(data_train.shape[1]).cuda()], MSWD_max_iter=MSWD_max_iter, verbose=verbose)

        layer.fit_spline(data_train[label_train, torch.arange(data_train.shape[1]).cuda()], label_train, edge_bins=edge_bins, derivclip=1, alpha=alpha, KDE=KDE, b_factor=b_factor, verbose=False)
        
        for label in range(nclass):
            data_train[label], logj_train1 = layer(data_train[label], torch.ones(data_train.shape[1], dtype=torch.int, device=data_train.device)*label)
            logj_train[label] = logj_train[label] + logj_train1

            data_valid[label], logj_valid1 = layer(data_valid[label], torch.ones(data_valid.shape[1], dtype=torch.int, device=data_valid.device)*label)
            logj_valid[label] = logj_valid[label] + logj_valid1

        model.add_layer(layer)


        logp_train = (torch.mean(logj_train[label_train, torch.arange(data_train.shape[1]).cuda()]) - ndim/2*torch.log(torch.tensor(2*math.pi)) - torch.mean(torch.sum(data_train[label_train, torch.arange(data_train.shape[1]).cuda()]**2,  dim=1)/2)).item()
        logp_valid = (torch.mean(logj_valid[label_valid, torch.arange(data_valid.shape[1]).cuda()]) - ndim/2*torch.log(torch.tensor(2*math.pi)) - torch.mean(torch.sum(data_valid[label_valid, torch.arange(data_valid.shape[1]).cuda()]**2,  dim=1)/2)).item()

        

        if logp_valid > best_logp_valid:
            best_logp_valid = logp_valid
            best_Nlayer = len(model.layer)
            wait = 0
        else:
            wait += 1
        if wait == maxwait:
            model.layer = model.layer[:best_Nlayer]
            break
        if ii>max_iter:
            break

        if verbose:
            if data_valid is not None: 
                print ('logp:', logp_train, logp_valid, 'time:', time.time()-t, 'iteration:', len(model.layer), 'best:', best_Nlayer)
            else:
                print ('logp:', logp_train, 'time:', time.time()-t, 'iteration:', len(model.layer))

        if iteration is not None and len(model.layer) >= iteration:
            if data_valid is not None:
                model.layer = model.layer[:best_Nlayer]
            break
        ii+=1

    return model

In [55]:
model = train_ConditionalGIS(train_x, train_y,valid_x, valid_y)

10 16
After whiten logp: -16.6646671295166 -15.997013092041016
Fit A: Time: 1.41450439453125 Wasserstein Distance: [1.2357678413391113, 1.0671168565750122, 0.9587216377258301, 0.921099841594696, 0.8842369914054871, 0.8027095794677734, 0.794550895690918, 0.6165199875831604]
logp: -13.777400970458984 -13.21005630493164 time: 5.357596397399902 iteration: 2 best: 2
Fit A: Time: 1.069533935546875 Wasserstein Distance: [1.1182522773742676, 0.9498315453529358, 0.9491696953773499, 0.8737191557884216, 0.7754488587379456, 0.709321916103363, 0.6868821978569031, 0.644119381904602]
logp: -11.873022079467773 -11.382253646850586 time: 5.0346269607543945 iteration: 3 best: 3
Fit A: Time: 1.1387740478515624 Wasserstein Distance: [0.9667471051216125, 0.922859787940979, 0.8313316702842712, 0.8187207579612732, 0.7010040283203125, 0.6983324289321899, 0.5883837342262268, 0.5799399614334106]
logp: -10.527931213378906 -10.099812507629395 time: 5.111476898193359 iteration: 4 best: 4
Fit A: Time: 1.382569946289

In [56]:
torch.save(model,os.path.join(data_path,'conditional_SINF_1d'))

In [23]:
print(data_path)

/global/cscratch1/sd/vboehm/SDSSOutlier/fc


In [24]:
model = torch.load(os.path.join(data_path,'conditional_SINF_1'))

In [25]:
_, num = np.unique(train['new_inf_labels'],return_counts=True)
log_prior = np.log(num/sum(num))

In [30]:
import scipy

In [31]:
print(scipy.special.logsumexp(log_prior))

0.0


In [32]:
train['initial_label_logprior']=log_prior

In [33]:
for data,dd  in zip([train_x,valid_x],[train,valid]):
    px = np.zeros((16,len(data)))
    for ii in range(16):
        px[ii]        = model.evaluate_density(data,ii*torch.ones((len(data))).to(device)).cpu().numpy()
    labels_new        = np.argmax(px,axis=0)
    dd['max_label_fc']  = labels_new
    dd['log_p_max_fc']  = px[labels_new,np.arange(len(labels_new))]
    dd['log_p_fc']      = px

In [34]:
_, num = np.unique(train['max_label_fc'],return_counts=True)
log_prior = np.log(num/sum(num))

train['fc_logprior'] = log_prior

In [35]:
for data,dd  in zip([train_x,valid_x],[train,valid]):
    dd['log_p_fc_marg']      = scipy.special.logsumexp(train['fc_logprior'][:,None]+dd['log_p_fc'], axis=0)
    dd['log_p_fc_marg_flat'] = scipy.special.logsumexp(dd['log_p_fc'], axis=0)


In [36]:
train.keys()

dict_keys(['spec', 'mask', 'noise', 'z', 'RA', 'DEC', 'class', 'subclass', 'SN', 'MJD', 'plate', 'fiber', 'mean', 'std', 'labels', 'new_inf_labels', 'initial_label_logprior', 'max_label_fc', 'log_p_max_fc', 'log_p_fc', 'fc_logprior', 'log_p_fc_marg', 'log_p_fc_marg_flat'])

In [37]:
pickle.dump(train,open(os.path.join(data_path,'SDSS_DR16_preprocessed_train_relabeled.pkl'),'wb'))
pickle.dump(valid,open(os.path.join(data_path,'SDSS_DR16_preprocessed_valid_relabeled.pkl'),'wb'))